# Getting Started

Agents use an LLM to determine which actions to take and in what order.
An action can either be using a tool and observing its output, or returning to the user.

When used correctly agents can be extremely powerful. The purpose of this notebook is to show you how to easily use agents through the simplest, highest level API.

In order to load agents, you should understand the following concepts:

- Tool: A function that performs a specific duty. This can be things like: Google Search, Database lookup, Python REPL, other chains. The interface for a tool is currently a function that is expected to have a string as an input, with a string as an output.
- LLM: The language model powering the agent.
- Agent: The agent to use. This should be a string that references a support agent class. Because this notebook focuses on the simplest, highest level API, this only covers using the standard supported agents. If you want to implement a custom agent, see the documentation for custom agents (coming soon).

**Agents**: For a list of supported agents and their specifications, see [here](agents.md).

**Tools**: For a list of predefined tools and their specifications, see [here](tools.md).

In [ ]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI
from langchain.tools import Tool
from langchain.utilities import PythonREPL

First, let's load the language model we're going to use to control the agent.

In [ ]:
llm = OpenAI(temperature=0)

Next, let's load some tools to use. Note that the `llm-math` tool uses an LLM, so we need to pass that in.

In [ ]:
def create_deployment_package(
    lambda_code, dependencies=[], project_name='lambda_project', output_zip_name='lambda.zip'
):
    """
    Create a deployment package with dependencies.
    """
    # Create the project directory
    os.makedirs(project_name, exist_ok=True)

    # Write the lambda code to the lambda_function.py file
    with open(os.path.join(project_name, "lambda_function.py"), "w") as f:
        f.write(lambda_code)

    # Install the dependencies to the package directory
    package_dir = os.path.join(project_name, "package")
    os.makedirs(package_dir, exist_ok=True)

    for dependency in dependencies:
        subprocess.run(["pip", "install", "--target", package_dir, dependency])

    # Create a .zip file for the deployment package
    with zipfile.ZipFile(output_zip_name, "w") as zipf:
        # Add the installed dependencies to the .zip file
        for root, _, files in os.walk(package_dir):
            for file in files:
                zipf.write(
                    os.path.join(root, file),
                    os.path.relpath(os.path.join(root, file), package_dir),
                )
        # Add the lambda_function.py file to the .zip file
        zipf.write(
            os.path.join(project_name, "lambda_function.py"), "lambda_function.py"
        )

    # Clean up the project directory
    #shutil.rmtree(project_name)

    return output_zip_name


In [ ]:
import boto3
import json
import os
import zipfile
import shutil
from botocore.exceptions import ClientError
import logging
import subprocess

client = boto3.client('lambda')

def create_lambda_function(lambda_data):
    """
    Create Lambda Function
    
    Input the the function if a stringified json object with the following fields:
        name - lambda function name
        code - lambda function code
        requirements - list of dependencies needed for the code to run
    """
    lines = lambda_data.split('\n')
    iam_role = os.getenv('LAMBDA_ROLE')
    meta = json.loads(lines[0])
    function_name = meta["name"]
    dependencies = meta["dependencies"]
    code = '\n'.join(lines[1:]) + '\n'
    print('code is ')
    print(code)
    
    # !!! HARD CODED !!!
    runtime = "python3.8"
    handler = "lambda_function.handler"

    output_zip_name = 'lambda.zip'
    create_deployment_package(
        code,
        dependencies=dependencies,
        output_zip_name=output_zip_name
    )
    
    try:

        with open(output_zip_name, 'rb') as zipfile:
            response = client.create_function(
                Code={
                    "ZipFile": zipfile.read()
                },
                FunctionName=function_name,
                Handler=handler,
                Runtime=runtime,
                Role=iam_role
            )

        cors_config = {
            "AllowMethods": ["*"],
            "AllowOrigins": ["*"],
            "AllowHeaders": ["access-control-allow-origin", "content-type"],
        }

        # Create a Function URL
        response = client.create_function_url_config(
            FunctionName=function_name,
            AuthType="NONE",
            Cors=cors_config,
        )
        logging.info("making URL public")

        # Make Function URL public
        # for some reason lambda:InvokeFunctionUrl doesnt exist in boto3 yet...
        cmd = [
            "aws",
            "lambda",
            "add-permission",
            "--function-name",
            function_name,
            "--action",
            "lambda:InvokeFunctionUrl",
            "--statement-id",
            "FunctionURLAllowPublicAccess",
            "--principal",
            "*",
            "--function-url-auth-type",
            "NONE",
        ]
        subprocess.run(cmd)

        logging.info("done and done")

        return json.dumps(response), 200
    except ClientError as e:
        return json.dumps(str(e)), e.response["Error"]["Code"]

In [ ]:
import boto3


def send_email(text):
    # Create a new SES resource
    print('sending email...', text)
    ses = boto3.client('ses')  # Replace with your desired region

    # Define the sender and recipient email addresses
    sender_email = 'mcclanahanbrian@gmail.com'
    recipient_email = 'mcclanahanbrian@gmail.com'

    # Create a MIME-formatted message
    subject = 'Langchain Example App'
    body_text = text
    body_html = '<html><body><h1>Chat GPT Example app</h1></body></html>'
    charset = 'UTF-8'

    # Create the message
    message = {
        'Subject': {
            'Data': subject,
            'Charset': charset
        },
        'Body': {
            'Text': {
                'Data': body_text,
                'Charset': charset
            }
        }
    }

    # Send the email
    response = ses.send_email(
        Source=sender_email,
        Destination={
            'ToAddresses': [recipient_email]
        },
        Message=message
    )

    # Print the MessageId of the sent email
    print('Email sent successfully. MessageId:', response['MessageId'])


In [ ]:
create_lambda_func_desc = """
    A tool used create lambda functions for Python 3.8. The input to the tool will be two or more lines. The first line will contain a json object which is properly enclosed using double quotes that has the name of the function under the field "name" and a list of dependencies under the field "dependencies". The second through the remaining lines will contain the code for the lambda function. The name of the lambda function must be 'handler'. Be sure to use proper python syntax when creating the lambda function. If the function needs to return html for a webpage then return a python dictionary with the "statusCode" field set to 200, set the "Content-Type" under "headers" with "text/html" and set "body" to be the html content.
"""

In [ ]:
tools = load_tools(
    ["serpapi"],
    llm=llm
)

In [ ]:
tools.extend([
    Tool.from_function(
        func=create_lambda_function,
        name = "createLambdaFunction",
        description=create_lambda_func_desc
    ),
    Tool.from_function(
        func=send_email,
        name = "sendEmail",
        description="A function to send emails. Input should be the content of the email."
    ),
])

In [ ]:
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True, max_iterations=3)
#agent = initialize_agent(tools, llm, agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True, max_iterations=3)

Now let's test it out!

In [ ]:
#agent.run("Create a lambda function named 'helloWorld' that returns 'Hello World'. Return the function url for the created lambda function.")

In [ ]:
#agent.run("create a lambda function named 'circleDragger' that returns the html for a website that lets me drag a red circle across the screen. Return the function url for the created lambda function.")

In [ ]:
from IPython.core.magic import register_line_cell_magic

In [ ]:
@register_line_cell_magic
def chat_agent(line, cell=None):
    "Magic that works both as %lcmagic and as %%lcmagic"
    if cell is None:
        response = agent.run(line)
        
    else:
        response = agent.run(cell)
    return response

In [ ]:
%%chat_agent
Create a lambda function that returns html with "hello Rikia how are you" in a header. The name of the lambda function should be testLambda. Return the function url for the created lambda function

In [ ]:
%%chat_agent
Send me an email that contains the weather in Oxon Hill Maryland